## 3D Implied Volatility Surface
Load `options_data.csv` and visualize implied volatility as a 3D scatter plot.

In [10]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

pd.options.display.float_format = lambda x: f'{x:.4f}'


In [11]:
CSV_PATH = 'options_data.csv'

df_raw = pd.read_csv(CSV_PATH)
required_cols = {'S0', 'K', 'T', 'C_mkt', 'iv'}
missing = required_cols - set(df_raw.columns)
if missing:
    raise ValueError(f'Input CSV is missing columns: {missing}')

spot = float(df_raw['S0'].median())
lower_bound = np.ceil((spot - 100.0) / 10.0) * 10.0
upper_bound = np.ceil((spot + 100.0) / 10.0) * 10.0
mask = (df_raw['K'] >= lower_bound) & (df_raw['K'] <= upper_bound)
df = df_raw.loc[mask].sort_values(['T', 'K']).reset_index(drop=True)
if df.empty:
    raise ValueError('No strikes within the specified window.')

print(
    f"Spot ~ {spot:.2f}. Keeping strikes in [{lower_bound:.2f}, {upper_bound:.2f}] => {len(df)} rows."
)


Spot ~ 671.93. Keeping strikes in [580.00, 780.00] => 489 rows.


In [12]:
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=df['K'],
            y=df['T'],
            z=df['iv'],
            mode='markers',
            marker=dict(
                size=5,
                color=df['iv'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title='IV'),
            ),
            text=[f"S0={s:.2f}<br>C_mkt={c:.2f}" for s, c in zip(df['S0'], df['C_mkt'])],
            hovertemplate='K=%{x:.2f}<br>T=%{y:.3f}<br>IV=%{z:.4f}<extra></extra>',
        )
    ]
)
fig.update_layout(
    title='Implied Volatility Scatter (S0 ± 100 filtered)',
    scene=dict(
        xaxis_title='Strike K',
        yaxis_title='Time to Maturity T (years)',
        zaxis_title='Implied Volatility',
    ),
    width=900,
    height=600,
)
fig.show()
